# RAG-Based Legal Contract Analyzer with CERT Monitoring

A **production-grade legal contract analysis system** using Gemini Pro with RAG (Retrieval-Augmented Generation), demonstrating contract risk identification, clause comparison, and compliance verification.

## Real-World Use Case

Legal teams and enterprises review thousands of contracts annually:
- **Vendor Agreements**: SaaS terms, service contracts
- **Employment Contracts**: Offer letters, NDAs
- **Partnership Agreements**: Joint ventures, licensing
- **Customer Contracts**: Terms of service, SLAs

Key challenges:
- Identifying risky clauses (indemnification, liability caps, termination)
- Comparing against company-approved templates
- Ensuring regulatory compliance (GDPR, SOC2, etc.)
- Tracking deviations from standard terms

## Architecture

```
Contract Document
       │
       ▼
┌─────────────────────────────────────────────────────────────┐
│  Document Chunker                                           │
│  → Split by sections, preserve clause boundaries           │
└─────────────────────────────────────────────────────────────┘
       │
       ▼
┌─────────────────────────────────────────────────────────────┐
│  Vector Store (Simulated RAG)                               │
│  → Store contract clauses + reference templates            │
│  → Semantic search for similar clauses                     │
└─────────────────────────────────────────────────────────────┘
       │
       ▼
┌─────────────────────────────────────────────────────────────┐
│  Gemini Pro - Risk Analysis                                 │
│  → Identify risk clauses                                   │
│  → Compare to standard templates                           │
│  → Generate recommendations                                │
└─────────────────────────────────────────────────────────────┘
       │
       ▼
┌─────────────────────────────────────────────────────────────┐
│  CERT Dashboard                                             │
│  • Track RAG retrieval quality                             │
│  • Grounding Check: Verify analysis is based on contract   │
│  • Audit trail for legal review process                    │
└─────────────────────────────────────────────────────────────┘
```

## Why Gemini Pro?

- **Long Context**: 1M+ tokens for entire contract suites
- **Reasoning**: Strong logical analysis for legal interpretation
- **Structured Output**: Reliable JSON schema adherence
- **Cost-Effective**: Lower cost than GPT-4 for high-volume processing

## 1. Setup & Installation

In [ ]:
# Install required packages
!pip install -q google-generativeai pydantic requests

print("Packages installed successfully!")

In [ ]:
import os
from getpass import getpass

# =============================================================
# API KEYS CONFIGURATION
# =============================================================

CERT_DASHBOARD_URL = "https://dashboard.cert-framework.com"

if 'CERT_API_KEY' not in os.environ:
    os.environ['CERT_API_KEY'] = getpass('Enter your CERT API Key: ')

if 'GOOGLE_API_KEY' not in os.environ:
    os.environ['GOOGLE_API_KEY'] = getpass('Enter your Google AI API key: ')

CERT_API_KEY = os.environ['CERT_API_KEY']
GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']

print(f"CERT Dashboard: {CERT_DASHBOARD_URL}")
print(f"CERT API Key: {CERT_API_KEY[:20]}...")
print(f"Google API Key: {GOOGLE_API_KEY[:20]}... (stays local)")

## 2. CERT Tracer with RAG Context Support

In [ ]:
import requests
import time
import uuid
from datetime import datetime
from typing import List, Optional, Dict, Any

class CERTTracer:
    """
    CERT Dashboard Tracer with RAG context support.
    
    For RAG applications, we send retrieved chunks as 'context'
    to enable verification that answers are grounded in retrieved content.
    """
    
    def __init__(self, dashboard_url: str, api_key: str, project_name: str = "default"):
        self.endpoint = f"{dashboard_url.rstrip('/')}/api/v1/traces"
        self.api_key = api_key
        self.project_name = project_name
        self.session_id = str(uuid.uuid4())[:8]
        self.traces_sent = 0
        self.total_tokens = 0
        
    def _get_headers(self) -> Dict[str, str]:
        return {
            "Content-Type": "application/json",
            "X-API-Key": self.api_key
        }
    
    def send_trace(self, trace_data: dict) -> bool:
        try:
            response = requests.post(
                self.endpoint,
                json={"traces": [trace_data]},
                headers=self._get_headers(),
                timeout=15
            )
            
            if response.status_code == 200:
                self.traces_sent += 1
                return True
            else:
                print(f"  [CERT] Error {response.status_code}")
                return False
                
        except Exception as e:
            print(f"  [CERT] Connection error: {e}")
            return False
    
    def trace_rag_query(
        self,
        model: str,
        query: str,
        response_text: str,
        duration_ms: float,
        prompt_tokens: int,
        completion_tokens: int,
        retrieved_chunks: List[str],
        analysis_type: str,
        contract_id: str = None,
        metadata: Optional[Dict[str, Any]] = None
    ) -> dict:
        """
        Record a RAG query trace with retrieved context.
        
        The retrieved_chunks parameter enables CERT to verify that
        the analysis is grounded in the actual contract content.
        """
        
        self.total_tokens += prompt_tokens + completion_tokens
        
        trace = {
            "id": f"{self.session_id}-{str(uuid.uuid4())[:8]}",
            "provider": "google",
            "model": model,
            "input": query,
            "output": response_text,
            "promptTokens": prompt_tokens,
            "completionTokens": completion_tokens,
            "durationMs": round(duration_ms, 2),
            "timestamp": datetime.utcnow().isoformat() + "Z",
            "context": retrieved_chunks,  # RAG retrieved content
            "metadata": {
                "project": self.project_name,
                "session_id": self.session_id,
                "analysis_type": analysis_type,
                "contract_id": contract_id,
                "chunks_retrieved": len(retrieved_chunks),
                **(metadata or {})
            }
        }
        
        self.send_trace(trace)
        return trace

# Initialize tracer
tracer = CERTTracer(
    dashboard_url=CERT_DASHBOARD_URL,
    api_key=CERT_API_KEY,
    project_name="legal-contract-analyzer"
)

print(f"CERT Tracer initialized")
print(f"  Session: {tracer.session_id}")
print(f"  RAG Context: ENABLED")

## 3. Sample Legal Contracts

Realistic contract examples representing common business agreements.

In [ ]:
# =============================================================
# SAMPLE CONTRACTS
# =============================================================

CONTRACTS = {
    "SAAS-2024-001": {
        "type": "SaaS Agreement",
        "vendor": "CloudTech Solutions Inc.",
        "content": """
MASTER SAAS SUBSCRIPTION AGREEMENT

This Master SaaS Subscription Agreement ("Agreement") is entered into as of November 1, 2024
("Effective Date") by and between:

PROVIDER: CloudTech Solutions Inc., a Delaware corporation
          123 Technology Drive, San Francisco, CA 94105
          
CUSTOMER: Acme Corporation
          456 Business Park, Austin, TX 78701

1. DEFINITIONS

1.1 "Services" means the cloud-based enterprise software platform known as "CloudTech Enterprise"
    and any associated support, maintenance, and professional services.

1.2 "Customer Data" means all data, content, and information uploaded, stored, or processed
    by Customer through the Services.

1.3 "Subscription Term" means the period specified in the Order Form during which Customer
    has access to the Services.

2. GRANT OF RIGHTS

2.1 License Grant. Subject to the terms of this Agreement, Provider grants Customer a
    non-exclusive, non-transferable right to access and use the Services during the
    Subscription Term solely for Customer's internal business purposes.

2.2 Restrictions. Customer shall not: (a) sublicense, sell, or transfer the Services;
    (b) modify, reverse engineer, or create derivative works; (c) use the Services
    in violation of applicable laws; (d) exceed the usage limits in the Order Form.

3. FEES AND PAYMENT

3.1 Fees. Customer shall pay the fees specified in the Order Form. All fees are
    non-refundable except as expressly set forth herein.

3.2 Payment Terms. Invoices are due within thirty (30) days of the invoice date.
    Late payments shall accrue interest at 1.5% per month or the maximum rate
    permitted by law, whichever is less.

3.3 Price Increases. Provider may increase fees upon renewal by providing at least
    sixty (60) days prior written notice. Fee increases shall not exceed 10% annually.

4. DATA PROTECTION AND SECURITY

4.1 Data Ownership. As between the parties, Customer retains all right, title, and
    interest in and to Customer Data.

4.2 Data Processing. Provider shall process Customer Data solely as necessary to
    provide the Services and in accordance with Customer's instructions.

4.3 Security Measures. Provider shall implement and maintain appropriate technical
    and organizational security measures, including:
    (a) SOC 2 Type II certification
    (b) AES-256 encryption at rest and TLS 1.3 in transit
    (c) Annual third-party penetration testing
    (d) 24/7 security monitoring

4.4 Data Breach Notification. Provider shall notify Customer of any confirmed data
    breach affecting Customer Data within 48 hours of discovery.

4.5 Data Location. Customer Data shall be stored and processed only in the United
    States and European Union, unless otherwise agreed in writing.

5. SERVICE LEVELS

5.1 Uptime Commitment. Provider commits to 99.9% monthly uptime for the Services.

5.2 Service Credits. If uptime falls below 99.9%, Customer shall receive service
    credits as follows:
    - 99.0% - 99.9%: 10% credit of monthly fees
    - 95.0% - 99.0%: 25% credit of monthly fees
    - Below 95.0%: 50% credit of monthly fees

5.3 Exclusions. The following are excluded from uptime calculations:
    (a) Scheduled maintenance (with 24 hours notice)
    (b) Emergency maintenance
    (c) Customer-caused issues
    (d) Force majeure events

6. WARRANTIES

6.1 Provider Warranties. Provider warrants that:
    (a) The Services will substantially conform to the documentation
    (b) Provider has the right to grant the licenses herein
    (c) The Services will not infringe third-party intellectual property rights

6.2 Disclaimer. EXCEPT AS EXPRESSLY SET FORTH HEREIN, PROVIDER DISCLAIMS ALL
    WARRANTIES, INCLUDING IMPLIED WARRANTIES OF MERCHANTABILITY, FITNESS FOR
    A PARTICULAR PURPOSE, AND NON-INFRINGEMENT.

7. LIMITATION OF LIABILITY

7.1 Liability Cap. EXCEPT FOR BREACHES OF SECTION 4 (DATA PROTECTION) OR GROSS
    NEGLIGENCE OR WILLFUL MISCONDUCT, NEITHER PARTY'S AGGREGATE LIABILITY SHALL
    EXCEED THE FEES PAID BY CUSTOMER IN THE TWELVE (12) MONTHS PRECEDING THE CLAIM.

7.2 Exclusion of Damages. IN NO EVENT SHALL EITHER PARTY BE LIABLE FOR INDIRECT,
    INCIDENTAL, SPECIAL, CONSEQUENTIAL, OR PUNITIVE DAMAGES, INCLUDING LOST
    PROFITS, REGARDLESS OF THE THEORY OF LIABILITY.

7.3 Data Protection Liability. Notwithstanding Section 7.1, Provider's liability
    for breaches of data protection obligations shall not exceed THREE TIMES (3X)
    the annual fees paid by Customer.

8. INDEMNIFICATION

8.1 Provider Indemnity. Provider shall indemnify, defend, and hold harmless Customer
    from any third-party claim that the Services infringe any patent, copyright,
    trademark, or trade secret.

8.2 Customer Indemnity. Customer shall indemnify, defend, and hold harmless Provider
    from any third-party claim arising from: (a) Customer Data; (b) Customer's use
    of the Services in violation of this Agreement; (c) Customer's violation of law.

8.3 Indemnification Procedures. The indemnified party shall: (a) promptly notify
    the indemnifying party; (b) grant sole control of defense and settlement;
    (c) provide reasonable cooperation.

9. TERM AND TERMINATION

9.1 Term. The initial Subscription Term is specified in the Order Form. Thereafter,
    the Agreement shall automatically renew for successive one (1) year periods
    unless either party provides written notice of non-renewal at least ninety
    (90) days prior to the end of the then-current term.

9.2 Termination for Cause. Either party may terminate this Agreement upon thirty
    (30) days written notice if the other party materially breaches this Agreement
    and fails to cure such breach within the notice period.

9.3 Termination for Convenience. Customer may terminate this Agreement for
    convenience upon ninety (90) days written notice, subject to payment of
    an early termination fee equal to 50% of the remaining Subscription Term fees.

9.4 Effect of Termination. Upon termination:
    (a) All rights granted to Customer shall immediately cease
    (b) Provider shall make Customer Data available for export for thirty (30) days
    (c) After the export period, Provider shall delete all Customer Data
    (d) Sections 7, 8, 10, and 11 shall survive termination

10. GOVERNING LAW AND DISPUTE RESOLUTION

10.1 Governing Law. This Agreement shall be governed by the laws of the State of
     Delaware, without regard to conflict of laws principles.

10.2 Dispute Resolution. Any dispute arising under this Agreement shall be resolved
     through binding arbitration in San Francisco, California, under the rules of
     JAMS. The arbitrator's decision shall be final and binding.

10.3 Equitable Relief. Notwithstanding Section 10.2, either party may seek
     injunctive or other equitable relief in any court of competent jurisdiction.

11. GENERAL PROVISIONS

11.1 Entire Agreement. This Agreement, together with any Order Forms, constitutes
     the entire agreement between the parties and supersedes all prior agreements.

11.2 Amendment. This Agreement may only be amended by a written instrument signed
     by authorized representatives of both parties.

11.3 Assignment. Neither party may assign this Agreement without the prior written
     consent of the other party, except that either party may assign to an
     affiliate or in connection with a merger, acquisition, or sale of all or
     substantially all of its assets.

11.4 Force Majeure. Neither party shall be liable for delays or failures in
     performance resulting from circumstances beyond its reasonable control.

IN WITNESS WHEREOF, the parties have executed this Agreement as of the Effective Date.

CLOUDTECH SOLUTIONS INC.           ACME CORPORATION

By: _________________________      By: _________________________
Name: John Smith                   Name: Jane Doe
Title: Chief Executive Officer     Title: Chief Technology Officer
Date: November 1, 2024             Date: November 1, 2024
"""
    },
    
    "NDA-2024-042": {
        "type": "Non-Disclosure Agreement",
        "vendor": "Strategic Partners LLC",
        "content": """
MUTUAL NON-DISCLOSURE AGREEMENT

This Mutual Non-Disclosure Agreement ("Agreement") is entered into as of November 15, 2024,
by and between:

Party A: TechVentures Inc., a California corporation
         789 Innovation Way, Palo Alto, CA 94301

Party B: Strategic Partners LLC, a Delaware limited liability company
         321 Enterprise Boulevard, Boston, MA 02110

(each a "Party" and collectively the "Parties")

1. PURPOSE

The Parties wish to explore a potential business relationship concerning joint development
of artificial intelligence solutions for the healthcare industry (the "Purpose"). In
connection with the Purpose, each Party may disclose Confidential Information to the other.

2. DEFINITION OF CONFIDENTIAL INFORMATION

2.1 "Confidential Information" means any non-public information disclosed by either Party
    to the other, including but not limited to:
    (a) Technical data, trade secrets, know-how, research, and product plans
    (b) Business information including customers, suppliers, and financial information
    (c) Software, source code, algorithms, and specifications
    (d) Personnel and organizational information
    (e) Any information marked as "Confidential" or disclosed under circumstances
        indicating its confidential nature

2.2 Confidential Information does not include information that:
    (a) Was publicly known at the time of disclosure
    (b) Becomes publicly known through no fault of the receiving Party
    (c) Was in the receiving Party's possession before disclosure
    (d) Is independently developed by the receiving Party without use of
        the disclosing Party's Confidential Information
    (e) Is rightfully obtained from a third party without restriction

3. OBLIGATIONS OF RECEIVING PARTY

3.1 Protection. The receiving Party shall:
    (a) Protect Confidential Information with at least the same degree of care used
        to protect its own confidential information, but no less than reasonable care
    (b) Not disclose Confidential Information to third parties without prior written consent
    (c) Limit access to employees and contractors with a need to know who are bound
        by confidentiality obligations at least as protective as this Agreement
    (d) Not use Confidential Information except for the Purpose

3.2 Compelled Disclosure. If the receiving Party is compelled by law to disclose
    Confidential Information, it shall:
    (a) Provide prompt written notice to the disclosing Party (unless prohibited by law)
    (b) Cooperate with the disclosing Party in seeking a protective order
    (c) Disclose only the minimum information legally required

4. INTELLECTUAL PROPERTY

4.1 Ownership. All Confidential Information remains the property of the disclosing Party.
    No license or other right to Confidential Information is granted except as expressly
    set forth in a separate written agreement.

4.2 Return of Materials. Upon request or termination of discussions, the receiving Party
    shall promptly return or destroy all tangible materials containing Confidential
    Information and certify such destruction in writing.

5. TERM AND TERMINATION

5.1 Term. This Agreement shall remain in effect for THREE (3) YEARS from the Effective
    Date, unless earlier terminated.

5.2 Survival. The confidentiality obligations shall survive termination and continue
    for FIVE (5) YEARS following disclosure of each item of Confidential Information.
    For trade secrets, obligations continue for as long as the information remains
    a trade secret under applicable law.

6. REMEDIES

6.1 Irreparable Harm. The Parties acknowledge that breach of this Agreement may cause
    irreparable harm for which monetary damages are inadequate. The disclosing Party
    shall be entitled to seek injunctive relief without the requirement of posting bond.

6.2 No Limitation. The remedies set forth herein are cumulative and in addition to
    any other remedies available at law or equity.

7. NO WARRANTY

ALL CONFIDENTIAL INFORMATION IS PROVIDED "AS IS" WITHOUT WARRANTY OF ANY KIND,
EXPRESS OR IMPLIED, INCLUDING WARRANTIES OF MERCHANTABILITY, FITNESS FOR A
PARTICULAR PURPOSE, OR NON-INFRINGEMENT.

8. NO OBLIGATION

Nothing in this Agreement shall obligate either Party to:
(a) Enter into any further agreement or business relationship
(b) Purchase or sell any products or services
(c) Disclose any particular Confidential Information

9. GENERAL PROVISIONS

9.1 Governing Law. This Agreement shall be governed by the laws of the State of
    New York, without regard to conflict of laws principles.

9.2 Entire Agreement. This Agreement constitutes the entire understanding between
    the Parties concerning the subject matter hereof.

9.3 Amendment. This Agreement may only be amended by written instrument signed by both Parties.

9.4 Counterparts. This Agreement may be executed in counterparts, each of which
    shall be deemed an original.

IN WITNESS WHEREOF, the Parties have executed this Agreement as of the Effective Date.

TECHVENTURES INC.                  STRATEGIC PARTNERS LLC

By: _________________________      By: _________________________
Name: Michael Chen                 Name: Sarah Johnson
Title: VP of Business Development  Title: Managing Partner
Date: November 15, 2024            Date: November 15, 2024
"""
    }
}

# Reference templates (company-approved standard terms)
REFERENCE_TEMPLATES = {
    "liability_cap": """
COMPANY STANDARD: Limitation of Liability

Preferred Terms:
- Liability cap: 12 months of fees paid (minimum acceptable)
- Data protection carve-out: 2x annual fees (maximum exposure)
- IP indemnification: Uncapped for provider
- Exclude indirect/consequential damages: Yes

Red Flags:
- Any cap less than 12 months fees
- No carve-out for data breaches
- Customer bears unlimited indemnity
- Gross negligence not carved out
""",
    "termination": """
COMPANY STANDARD: Termination Rights

Preferred Terms:
- Termination for convenience: 30 days notice
- Termination for cause: 30 days cure period
- No early termination fees
- Data export: 60 days minimum
- Pro-rata refund on termination

Red Flags:
- Early termination fees > 25% remaining value
- Auto-renewal without 60+ days notice
- Data deletion < 30 days
- No cure period for breach
""",
    "data_protection": """
COMPANY STANDARD: Data Protection

Required Terms:
- SOC 2 Type II certification
- Encryption at rest (AES-256) and in transit (TLS 1.2+)
- Breach notification: 72 hours maximum
- Data location: US or EU only (explicit agreement for others)
- Annual security audits
- DPA/GDPR compliance

Red Flags:
- No security certifications
- Breach notification > 72 hours
- Unspecified data location
- No deletion upon termination
""",
    "nda_standard": """
COMPANY STANDARD: NDA Terms

Preferred Terms:
- Mutual NDA (not one-way)
- Term: 2-3 years
- Survival: 3-5 years (not perpetual)
- Trade secrets: Perpetual protection OK
- Clear exceptions for public info, prior knowledge, independent development
- Compelled disclosure protection

Red Flags:
- Perpetual survival for all information
- Overly broad definition of confidential information
- Non-compete provisions disguised as NDA
- Unreasonable geographic restrictions
- Assignment to third parties
"""
}

print(f"Loaded {len(CONTRACTS)} contracts and {len(REFERENCE_TEMPLATES)} reference templates")

## 4. Data Models for Legal Analysis

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional
from enum import Enum
import json

class RiskLevel(str, Enum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"
    CRITICAL = "critical"

class ClauseAnalysis(BaseModel):
    clause_name: str = Field(description="Name of the clause")
    section_reference: str = Field(description="Section number in contract")
    clause_text: str = Field(description="Exact text from contract")
    risk_level: str = Field(description="low, medium, high, or critical")
    risk_explanation: str = Field(description="Why this is risky")
    deviation_from_standard: Optional[str] = Field(description="How it differs from company standard")
    recommended_changes: List[str] = Field(description="Suggested modifications")

class ContractSummary(BaseModel):
    contract_type: str = Field(description="Type of contract")
    parties: List[str] = Field(description="Parties to the contract")
    effective_date: str = Field(description="When contract takes effect")
    term_length: str = Field(description="Duration of contract")
    total_value: Optional[str] = Field(description="Financial value if applicable")
    key_obligations: List[str] = Field(description="Main obligations for each party")

class RiskAssessment(BaseModel):
    overall_risk: str = Field(description="Overall risk level")
    risk_score: int = Field(description="1-100 risk score")
    high_risk_clauses: List[ClauseAnalysis] = Field(description="Clauses requiring attention")
    missing_protections: List[str] = Field(description="Standard protections not present")
    recommendations: List[str] = Field(description="Prioritized list of recommended actions")
    proceed_recommendation: str = Field(description="Sign as-is, negotiate, or reject")

print("Legal analysis data models defined")

## 5. Gemini Pro Analysis Functions

In [ ]:
import google.generativeai as genai

# Configure Gemini
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-pro')

def analyze_contract_risks(
    contract_id: str,
    contract: dict,
    templates: dict,
    tracer: CERTTracer
) -> dict:
    """
    Analyze a contract for legal risks and deviations from company standards.
    
    Uses RAG pattern: retrieves relevant template sections and analyzes
    contract clauses against them.
    """
    
    print(f"\n{'='*60}")
    print(f"ANALYZING CONTRACT: {contract_id}")
    print(f"Type: {contract['type']}")
    print(f"{'='*60}")
    
    contract_text = contract['content']
    
    # Simulate RAG retrieval - get relevant reference templates
    retrieved_templates = []
    if "SaaS" in contract['type']:
        retrieved_templates = [
            templates['liability_cap'],
            templates['termination'],
            templates['data_protection']
        ]
    elif "NDA" in contract['type'] or "Non-Disclosure" in contract['type']:
        retrieved_templates = [
            templates['nda_standard']
        ]
    
    print(f"Retrieved {len(retrieved_templates)} reference templates")
    
    # Build RAG context
    rag_context = [contract_text] + retrieved_templates
    
    analysis_prompt = f"""You are an expert corporate attorney reviewing a contract for legal risks.

TASK: Analyze the following contract and identify all risky clauses, comparing them against
the company's standard terms provided as reference.

CONTRACT TO ANALYZE:
{contract_text}

COMPANY STANDARD TERMS (Reference):
{''.join(retrieved_templates)}

ANALYSIS REQUIREMENTS:
1. Identify each clause that deviates from company standards or poses risk
2. Quote the exact clause text from the contract
3. Explain why each clause is risky
4. Provide specific recommended changes
5. Assess overall contract risk

Return a JSON response with this structure:
{{
    "contract_summary": {{
        "contract_type": "...",
        "parties": ["Party A", "Party B"],
        "effective_date": "...",
        "term_length": "...",
        "key_obligations": ["..."]
    }},
    "risk_assessment": {{
        "overall_risk": "low|medium|high|critical",
        "risk_score": 0-100,
        "high_risk_clauses": [
            {{
                "clause_name": "...",
                "section_reference": "Section X.X",
                "clause_text": "exact quote from contract",
                "risk_level": "low|medium|high|critical",
                "risk_explanation": "...",
                "deviation_from_standard": "...",
                "recommended_changes": ["..."]
            }}
        ],
        "missing_protections": ["..."],
        "recommendations": ["prioritized action items"],
        "proceed_recommendation": "Sign as-is | Negotiate changes | Do not sign"
    }}
}}"""
    
    start_time = time.time()
    
    response = model.generate_content(
        analysis_prompt,
        generation_config=genai.GenerationConfig(
            temperature=0.1,
            max_output_tokens=4096
        )
    )
    
    duration_ms = (time.time() - start_time) * 1000
    output_text = response.text
    
    # Get token counts
    prompt_tokens = model.count_tokens(analysis_prompt).total_tokens
    completion_tokens = model.count_tokens(output_text).total_tokens
    
    print(f"\nAnalysis completed in {duration_ms/1000:.1f}s")
    print(f"Tokens: {prompt_tokens:,} input, {completion_tokens:,} output")
    
    # Send trace to CERT with RAG context
    tracer.trace_rag_query(
        model="gemini-1.5-pro",
        query=f"Analyze contract {contract_id} for legal risks",
        response_text=output_text,
        duration_ms=duration_ms,
        prompt_tokens=prompt_tokens,
        completion_tokens=completion_tokens,
        retrieved_chunks=rag_context,  # RAG context for Grounding Check
        analysis_type="risk_analysis",
        contract_id=contract_id
    )
    
    # Parse JSON response
    try:
        # Find JSON in response
        json_start = output_text.find('{')
        json_end = output_text.rfind('}') + 1
        return json.loads(output_text[json_start:json_end])
    except json.JSONDecodeError:
        return {"raw_response": output_text}

print("Contract risk analysis function ready")

In [ ]:
def compare_clauses(
    contract_id: str,
    contract: dict,
    clause_type: str,
    reference_template: str,
    tracer: CERTTracer
) -> dict:
    """
    Deep-dive comparison of a specific clause type against company standards.
    """
    
    print(f"\n{'='*60}")
    print(f"CLAUSE COMPARISON: {clause_type}")
    print(f"{'='*60}")
    
    contract_text = contract['content']
    
    comparison_prompt = f"""You are a legal analyst comparing contract clauses.

TASK: Extract and compare the {clause_type} provisions in this contract
against the company's standard terms.

CONTRACT:
{contract_text}

COMPANY STANDARD:
{reference_template}

Provide a detailed comparison in JSON format:
{{
    "clause_type": "{clause_type}",
    "contract_terms": {{
        "section_reference": "Section X.X",
        "exact_text": "quote from contract",
        "key_provisions": ["list of key points"]
    }},
    "comparison": {{
        "meets_standard": true/false,
        "deviations": [
            {{
                "issue": "description of deviation",
                "contract_says": "what contract states",
                "standard_says": "what company standard requires",
                "risk_impact": "business impact of this deviation"
            }}
        ],
        "missing_provisions": ["provisions from standard not in contract"]
    }},
    "negotiation_points": [
        {{
            "issue": "what to negotiate",
            "suggested_language": "proposed replacement text",
            "priority": "high|medium|low"
        }}
    ]
}}"""
    
    start_time = time.time()
    
    response = model.generate_content(
        comparison_prompt,
        generation_config=genai.GenerationConfig(
            temperature=0.1,
            max_output_tokens=3000
        )
    )
    
    duration_ms = (time.time() - start_time) * 1000
    output_text = response.text
    
    prompt_tokens = model.count_tokens(comparison_prompt).total_tokens
    completion_tokens = model.count_tokens(output_text).total_tokens
    
    print(f"Comparison completed in {duration_ms/1000:.1f}s")
    
    # Send trace to CERT
    tracer.trace_rag_query(
        model="gemini-1.5-pro",
        query=f"Compare {clause_type} clauses in {contract_id}",
        response_text=output_text,
        duration_ms=duration_ms,
        prompt_tokens=prompt_tokens,
        completion_tokens=completion_tokens,
        retrieved_chunks=[contract_text, reference_template],
        analysis_type="clause_comparison",
        contract_id=contract_id,
        metadata={"clause_type": clause_type}
    )
    
    try:
        json_start = output_text.find('{')
        json_end = output_text.rfind('}') + 1
        return json.loads(output_text[json_start:json_end])
    except json.JSONDecodeError:
        return {"raw_response": output_text}

print("Clause comparison function ready")

## 6. Analyze Contracts

In [ ]:
# Analyze SaaS Agreement
saas_contract_id = "SAAS-2024-001"
saas_contract = CONTRACTS[saas_contract_id]

saas_analysis = analyze_contract_risks(
    saas_contract_id,
    saas_contract,
    REFERENCE_TEMPLATES,
    tracer
)

print("\n" + "-"*60)
print("RISK ANALYSIS RESULTS:")
print("-"*60)
print(json.dumps(saas_analysis, indent=2))

In [ ]:
# Deep-dive on liability clauses
liability_comparison = compare_clauses(
    saas_contract_id,
    saas_contract,
    "Limitation of Liability",
    REFERENCE_TEMPLATES['liability_cap'],
    tracer
)

print("\n" + "-"*60)
print("LIABILITY CLAUSE COMPARISON:")
print("-"*60)
print(json.dumps(liability_comparison, indent=2))

In [ ]:
# Deep-dive on termination clauses
termination_comparison = compare_clauses(
    saas_contract_id,
    saas_contract,
    "Termination",
    REFERENCE_TEMPLATES['termination'],
    tracer
)

print("\n" + "-"*60)
print("TERMINATION CLAUSE COMPARISON:")
print("-"*60)
print(json.dumps(termination_comparison, indent=2))

In [ ]:
# Analyze NDA
nda_contract_id = "NDA-2024-042"
nda_contract = CONTRACTS[nda_contract_id]

nda_analysis = analyze_contract_risks(
    nda_contract_id,
    nda_contract,
    REFERENCE_TEMPLATES,
    tracer
)

print("\n" + "-"*60)
print("NDA RISK ANALYSIS:")
print("-"*60)
print(json.dumps(nda_analysis, indent=2))

## 7. Generate Negotiation Memo

In [ ]:
def generate_negotiation_memo(
    contract_id: str,
    contract: dict,
    risk_analysis: dict,
    tracer: CERTTracer
) -> str:
    """
    Generate a negotiation memo for the legal team.
    """
    
    print(f"\n{'='*60}")
    print(f"GENERATING NEGOTIATION MEMO: {contract_id}")
    print(f"{'='*60}")
    
    memo_prompt = f"""You are a senior corporate attorney preparing a negotiation memo.

Based on the following contract and risk analysis, prepare a professional
negotiation memo for the legal team.

CONTRACT:
{contract['content'][:5000]}  # First 5000 chars for context

RISK ANALYSIS:
{json.dumps(risk_analysis, indent=2)}

Generate a negotiation memo with the following sections:

1. EXECUTIVE SUMMARY (2-3 sentences)
2. RECOMMENDATION (Sign as-is / Negotiate / Do not sign)
3. KEY ISSUES REQUIRING NEGOTIATION (prioritized list)
4. PROPOSED REDLINES (specific language changes)
5. DEAL BREAKERS (issues that must be resolved)
6. ACCEPTABLE COMPROMISES (where we can be flexible)

Write in professional legal memo format."""
    
    start_time = time.time()
    
    response = model.generate_content(
        memo_prompt,
        generation_config=genai.GenerationConfig(
            temperature=0.3,
            max_output_tokens=2000
        )
    )
    
    duration_ms = (time.time() - start_time) * 1000
    memo_text = response.text
    
    prompt_tokens = model.count_tokens(memo_prompt).total_tokens
    completion_tokens = model.count_tokens(memo_text).total_tokens
    
    print(f"Memo generated in {duration_ms/1000:.1f}s")
    
    # Send trace to CERT
    tracer.trace_rag_query(
        model="gemini-1.5-pro",
        query=f"Generate negotiation memo for {contract_id}",
        response_text=memo_text,
        duration_ms=duration_ms,
        prompt_tokens=prompt_tokens,
        completion_tokens=completion_tokens,
        retrieved_chunks=[contract['content'], json.dumps(risk_analysis)],
        analysis_type="negotiation_memo",
        contract_id=contract_id
    )
    
    return memo_text

# Generate memo for SaaS contract
saas_memo = generate_negotiation_memo(
    saas_contract_id,
    saas_contract,
    saas_analysis,
    tracer
)

print("\n" + "-"*60)
print("NEGOTIATION MEMO:")
print("-"*60)
print(saas_memo)

## 8. CERT Dashboard Summary

In [ ]:
print("="*60)
print("CERT DASHBOARD - SESSION SUMMARY")
print("="*60)
print(f"\nSession ID: {tracer.session_id}")
print(f"Project: {tracer.project_name}")
print(f"Traces sent: {tracer.traces_sent}")
print(f"Total tokens: {tracer.total_tokens:,}")

print(f"\n{'─'*60}")
print("RAG ANALYSIS TRACES")
print(f"{'─'*60}")
print("\nEach trace includes:")
print("  - Contract content (source document)")
print("  - Retrieved reference templates (RAG context)")
print("  - LLM analysis output")
print("\nThis enables:")
print("  1. Grounding Check - Verify analysis cites actual contract text")
print("  2. Quality Metrics - Track analysis accuracy over time")
print("  3. Audit Trail - Full traceability for legal review")

print(f"\n{'─'*60}")
print("CONTRACTS ANALYZED")
print(f"{'─'*60}")
for contract_id, contract in CONTRACTS.items():
    print(f"  - {contract_id}: {contract['type']}")

print(f"\nView traces at: {CERT_DASHBOARD_URL}/quality")
print("="*60)

---

## Summary

### What This Notebook Demonstrates

| Feature | Implementation |
|---------|----------------|
| **RAG Pattern** | Retrieved reference templates for clause comparison |
| **Risk Analysis** | Gemini Pro identifies risky clauses with citations |
| **Clause Comparison** | Detailed deviation analysis against company standards |
| **Memo Generation** | Professional negotiation memo with redlines |
| **Audit Trail** | Full traceability of all analysis for legal review |

### CERT Integration for Legal AI

Key integration points:

```python
tracer.trace_rag_query(
    model="gemini-1.5-pro",
    query="Analyze contract...",
    response_text=analysis_result,
    retrieved_chunks=[contract_text, template1, template2],  # RAG context
    analysis_type="risk_analysis",
    contract_id="CONTRACT-001"
)
```

### Production Considerations

1. **Vector Store**: Replace simulated RAG with actual vector DB (Pinecone, Weaviate)
2. **Document Processing**: OCR + PDF extraction for scanned contracts
3. **Clause Library**: Build comprehensive template library over time
4. **Human Review**: Flag high-risk analyses for attorney review
5. **Version Control**: Track contract versions and amendments
6. **Integration**: Connect to contract management systems (DocuSign, Ironclad)